## Scenario Filtering
### Alison Campion and Alex Gottlieb

This notebook is a guide to using the geography and time filtering associated with defining a scenario to run the DARPA World Modeleres system.  The front end allows the user to select scenario parameters.  For now, we will look at the geography and time parameters, although there are any number of variables that can be set to specific values or set within contrained bounds to reflect the desired scenario.  

Geography and time are set using the front end and passed to the model pipelines. The pipelines then use that information to mask or filter the data that is being used in the models to run the model to the user's scenario specification.  To be clear, at this point the models have been pre-trained using the full dataset.  The scenario parameters simply limit the data that is run through the model pipeline to get the results to the question posed by the user.

### Restructuring Pipelines
To most efficiently mask data to the time and geography of a scenario, we are requiring a task that writes all properly preprocessed data (see **Metadata and Data Requirements** below) to a single directory, *before* merging data into the data frame or array that will be used to run the model. We recommend splitting each pipeline into 2 parts: one preprocessing file that ends with a data consolidation task and one modeling file that contains the tasks used to train the model and make predictions for the scenario. This results in cleaner code with clear boundaries. For example, the market price model was split into `data_cleaning.py` and `tasks.py` (although the latter could probably be renamed to something more descriptive). The terminal task of `data_cleaning.py` requires all upstream preprocessing tasks and moves all that data to a single directory stored in an `IntermediateTarget`, as follows:

In [ ]:
@requires(
    PrepareSouthSudanPriceData,
    PrepareFatalitiesData,
    PrepareRainfallData,
    PrepareCropProductionData,
)
class WriteCleanedPriceModelData(Task):
    """
    This task consolidates all of the data used as input to the price model and writes it to a single directory.
    Requires all preprocessing tasks.
    """
    def output(self):
        return IntermediateTarget(path=f"{self.task_id}/", timeout=3600)

    def run(self):
        with self.output().temporary_path() as tmpdir:
            os.makedirs(tmpdir)
            for prep_task in self.input():
                with prep_task.open("r") as src:
                    d = src.read()
                    for k, v in d.items():
                        # write vector files
                        if isinstance(v, gpd.GeoDataFrame):
                            schema = datetime_schema(v)
                            v.to_file(
                                os.path.join(tmpdir, f"{k}.geojson"),
                                driver="GeoJSON",
                                schema=schema,
                            )
                        # write rasters (actually, just copy them over)
                        elif isinstance(v, str) and v.endswith("tif"):
                            shutil.copy(v, os.path.join(tmpdir, k))
                        else:
                            raise TypeError(
                                "Object must be GeoDataFrame or path to .tif file"
                            )

The modeling pipeline, then, begins by masking the data to the time and geography that will be used to generate the scenario predictions. We have written flexible tasks located in `utils/scenario_tasks/functions` for performing this masking called `MaskDataToTime` and `MaskDataToGeography`, to be used sequentially in that order for efficiency's sake. The first masking task requires the terminal data consolidation task of the cleaning pipeline and takes the `DateIntervalParameter` `time` which is a range in the form `"start_date-end_date"` (`end_date` is not inclusive!), where each date is in iso format `"YYYY-MM-DD"`, so for example `"2019-01-01-2019-02-01"` for all dates from January 1st up to January 31st (*note that this is subject to change depending on how time is ultimately handled on the front end*).
Setting a default value for the `DateIntervalParameter` in a Luigi Task requires a `luigi.date_interval.Custom` object (which is what the `"start_date-end_date"` value will be parsed to), e.g.:

```
time = DateIntervalParameter(
    default=Custom(
        datetime.date.fromisoformat("2009-01-01"),
        datetime.date.fromisoformat("2018-01-01"),
    )
)
```

`MaskDataToGeography` takes the `GeoParameter` `geography`, which is a GeoJSON-like dictionary. Put together, it looks like this:

In [ ]:
import datetime

from luigi.date_interval import Custom as CustomDateInterval
from luigi.parameter import DateIntervalParameter
from luigi.util import requires

from kiluigi.parameter import GeoParameter
from models.market_price_model.data_cleaning import WriteCleanedPriceModelData
from utils.scenario_tasks.functions.geography import MaskDataToGeography
from utils.scenario_tasks.functions.time import MaskDataToTime


@requires(WriteCleanedPriceModelData)
class MaskPriceModelDataToTime(MaskDataToTime):

    time = DateIntervalParameter(
        default=CustomDateInterval(
            datetime.date.fromisoformat("2009-01-01"),
            datetime.date.fromisoformat("2018-01-01"),
        )
    )


@requires(MaskPriceModelDataToTime)
class MaskPriceModelDataToGeography(MaskDataToGeography):

    geography = GeoParameter(
        default={
            "coordinates": [
                [
                    [31.598_289_489_746_2, 3.689_588_069_915_83],
                    [31.590_587_615_966_9, 3.687_186_956_405_64],
                    ...
                    [31.608_896_255_493_1, 3.692_878_961_563_17],
                    [31.598_289_489_746_2, 3.689_588_069_915_83],
                ]
            ],
            "type": "Polygon",
        }
    )


From there, you now have the raw data to run your scenario for the desired time and geography, stored in `output/IntermediateTargets/MaskPriceModelDataToGeography/.pickle/` (as well as the raw data to train the model in `output/IntermediateTargets/WriteCleanedPriceModelData/.pickle/`). The next step would be to write a task to merge these files into whatever format you need to train or test your model.


### Metadata and Data Requirements
1. Vector files must be written out in `.geojson` format. Most analysts work with vector data using the `geopandas.GeoDataFrame` object and save it using the `.to_file()` method. The default driver for the method is an ESRI Shapefile, but Shapefiles limit the length of field names to 10 characters, so any variable names longer than that limit will be truncated. Therefore, all analysts should use the `driver="GeoJSON"` argument in `.to_file()`.

2. All vector files must have `Time` and `geometry` properties. The former is a `datetime.datetime` object denoting the time at which the observation was recorded and the latter is a `shapely.geometry` object (probably a `Polygon`) denoting the geographic extent of the observation.

3. The GeoJSON driver does have one catch, in that it doesn't inherently recognize `datetime` columns when you are writing a `GeoDataFrame`. Accordingly, you need to manually update the schema, using the following function (currently located in `models/market_price/model/utils.py`, but subject to move):


In [2]:
def datetime_schema(gdf):
    schema = gpd.io.file.infer_schema(gdf)
    schema["properties"]["Time"] = "datetime"
    return schema

4. Rasters must have a `Time` field in the `tags()` attribute of the Raster. This is how the `MaskDataToTime` task can read the time associated with the raster.  When the raster is saved to file the `Time` attribute must be saved as a string, so the field should be a string date formatted in the same way as the `DateIntervalParameter` (`YYY-MM-DD`).  To acccess the `tags()` attribute and set the `Time` field you can do the following:


    with rasterio.open(raster_file) as src:
        tags = src.tags()
        tags['Time'] = '2019-01-01'

   The raster files that are written out after cleaning and assembly should already have this `Time` attribute added.

5. The Time filter does allow files to be geojson, raster, json, excel, csv, or a pickled dataframe.  However after time filtering, the files will need to be georeferenced (raster or geojson) to be filtered for geography.

### Warnings and Assert Messages

There are many built-in warnings and assert messages in the Time and Geography filters.  If you get a warning or an assert message, trace it back in the code to find out what is wrong with your data.  Checks and warnings include:

1. Making sure the filepath or folder path with the data exists and has files in it.
2. Making sure the files are in a format that can be read by the filters.
3. Are `Time` and `Geography` included as columns or attributes in eac data file appropriately.
4. Is the Mask time or space within the bounds of the data you are passing? For example, you may pass rainfall data from January to May 2018 but pass a Time Mask of June to July 2018.  In this case there is no overlap between the mask and your data so the filter will return a warning and not pass any rainfall data onto the filtered stage.

### Full Scenario Filtering

Moving forward, we need to filter beyond just geography and time.  Each parameter set from the front end is read in and saved as a json file.  This json file includes the name of the task being run, a list of parameters, and a geojson shape.  In the future, it should also contain a time window.  Each parameter also then has attributes.  Here is an example of the json file coming out of the front end right now.  Note that I have shortened the geojson and list of parameters for the sake of demonstration.

In [10]:
samplePayload = {
    "name": "models.malnutrition_model.tasks.Raster2CSVToCkan",
    "registry_task_url": "models__malnutrition_model__tasks__Raster2CSVToCkan",
    "parameters": [
      {
        "name": "PercentOfNormalRainfall",
        "description": "",
        "default": 0.5,
        "parameter_class": "luigi.parameter.FloatParameter"
      },
    ],
    "geoJSON":  {"coordinates":[[[29.9418830871583,10.2875909805297],
                                 [29.9422702789307,10.2877197265626],
                                 [29.9967708587646,10.2888002395631]]],
                 "type":"Polygon","properties":{"name":{},"lastUpdated":"2019-03-11T15:04:58.442Z"}},
    "is_terminal_task": True
  }

This file will be saved as a descriptor of the scenario so that results can be tied back to all the information you would need to understand what scenario was run.  We need to make some changes to this structure to allow it to reflect the range of scenarios we would want to run.  A scenario can be defined as any set of constraints or conditions the user applies to variables (or parameters) contained in the model.  Each variable should be allowed to be varied or changed over a certain time and space.

Let's use rainfall as an example.  Say I want to understand how 2 months of low rainfall during the planting season will affect the harvest or even the subsequent lean season. I may want to run the scenario for 1 year, but only apply 2 months of low rainfall. I also may want to apply low rainfall in only specific locations, perhaps the highly productive agricultural regions.  In order to capture this scenario information, we need to now the time and spatial extent of the entire scenario as well as the time and spatial extent of the variables we want to modify, which in this case is only rainfall. But you can imagine that a user may want to chain a set of events that occur at different times or in different locations.  

Here is an example of the type of json input that would match this scenario definition.

In [11]:
samplePayload = {
    "name": "models.malnutrition_model.tasks.Raster2CSVToCkan",
    "registry_task_url": "models__malnutrition_model__tasks__Raster2CSVToCkan",
    "parameters": [
      {
        "name": "rainfall",
        "description": "",
        "default": 0.5,
        "condition": 0.2,
        "parameter_class": "luigi.parameter.FloatParameter",
        "Time": "2018-05-01-2018-08-01",
        "geoJSON":  {"coordinates":[[[29.9418830871583,10.2875909805297],
                                     [29.9422702789307,10.2877197265626],
                                     [29.9967708587646,10.2888002395631]]],
                     "type":"Polygon","properties":{"name":{},"lastUpdated":"2019-03-11T15:04:58.442Z"}},
      },
    ],
    "Time": "2018-03-01-2019-03-01",
    "geoJSON":  {"coordinates":[[[29.9418830871583,10.2875909805297],
                                 [29.9422702789307,10.2877197265626],
                                 [29.9967708587646,10.2888002395631]]],
                 "type":"Polygon","properties":{"name":{},"lastUpdated":"2019-03-11T15:04:58.442Z"}},
    "is_terminal_task": True
  }

A few important changes to note:

1. The name of the parameter is `'rainfall'`. It should match the variable name, or be able to be matched to the standard variable name, that we use in our pipelines. 
2. The `'condition'` attribute is how the variable will be modulated.  This may be as simple as being a multiplicative factor or threshold, or it may be as complicated as requiring historical data to be pulled to create a distribution that is then sampled at a specific level to synthetically create "low" or "high" input (i.e. for rainfall or whether variables).  For now, we will keep the condition simple but in the future that condition might be associated with a class of methods that can apply certain transformations that would be necessary to create the input data for the scenario.
3. There is a `'geoJSON'` and `'Time'` parameter for bothe the entire scenario (outside of the parameter list) and for the parameter.

We envision that all of the input data will be masked to the scenario duration and spatial extent, then each variables will be modified according to the user input conditions to execute the scenario.

### ScenarioParameter and ScenarioDefinitionTask

We are writing the `ScenarioParameter` to import the json file coming in from the front end.  The benefit of using a parameter means that the entire scenario can be passed through the pipeline. Then we will write a `ScenarioDefinitionTask` that will contain methods to parse the data from the json structure and apply the variable conditions to the raw data to generate the scenario data.

We are open to other solutions for applying the scenario paramaterization, perhaps using a `ScenarioClass` that encompasses both the json structure and the methods to apply the variable conditions. 